In [0]:
import numpy as np
import pandas as pd
import matplotlib.image as img
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd drive/My\ Drive/465data/

/content/drive/My Drive/465data


In [7]:
ls

32ptx_data_n                         HAM10000_images_part_2labels_np.npy
HAM10000_images_part_2images_np.npy  images_np_32.npy


In [0]:
dataset = pd.read_csv("32ptx_data_n")


In [11]:
print("Shape of dataset", dataset.shape)
dataset.head()

Shape of dataset (9993, 3080)


,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,3039,3040,3041,3042,3043,3044,3045,3046,3047,3048,3049,3050,3051,3052,3053,3054,3055,3056,3057,3058,3059,3060,3061,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071,image_id,lesion_id,dx,dx_type,age,sex,localization
0,0,229,128,132,232,126,128,230,128,127,233,131,133,238,134,141,237,133,141,238,136,145,233,132,135,237,132,139,242,141,157,244,143,156,244,151,160,245,150,160,...,228,140,137,231,144,138,227,141,130,224,140,129,227,145,133,225,142,128,224,144,130,223,148,136,224,150,139,222,150,138,215,145,133,ISIC_0024306,HAM_0000550,nv,follow_up,45.0,male,trunk
1,1,216,122,134,217,120,132,217,117,133,218,120,133,215,116,123,216,118,124,218,122,134,222,127,142,225,133,148,221,128,143,227,134,153,234,143,162,233,143,162,...,226,144,150,226,144,151,222,143,151,224,146,153,223,147,154,218,143,149,217,142,148,219,144,150,213,141,145,214,140,145,208,136,139,ISIC_0024307,HAM_0003577,nv,follow_up,50.0,male,lower extremity
2,2,245,171,177,246,173,180,247,174,182,247,175,182,248,180,187,248,181,188,248,179,185,248,180,186,248,182,188,248,182,187,248,182,186,248,182,186,248,185,188,...,229,158,162,228,156,158,225,150,147,223,145,145,221,145,147,215,136,138,215,136,140,213,136,144,213,139,149,217,147,155,215,144,154,ISIC_0024308,HAM_0001477,nv,follow_up,55.0,female,trunk
3,3,209,121,135,209,118,134,210,122,137,211,125,140,212,123,139,212,121,138,213,123,140,216,133,146,218,135,151,217,126,143,217,127,138,222,139,148,224,145,156,...,197,120,130,192,117,127,191,118,126,196,122,127,198,127,127,199,129,127,197,126,121,195,123,119,193,121,118,190,120,120,189,122,125,ISIC_0024309,HAM_0000484,nv,follow_up,40.0,male,trunk
4,4,64,28,34,92,45,50,190,129,134,210,151,157,210,141,159,212,147,159,215,161,170,217,160,174,214,150,153,220,169,177,214,158,149,208,140,122,216,153,150,...,215,150,156,221,167,187,214,152,156,205,123,104,213,149,154,208,144,131,203,126,108,207,135,138,206,137,141,140,85,85,69,30,41,ISIC_0024310,HAM_0003350,mel,histo,60.0,male,chest


In [23]:
X = dataset.drop(columns = ["Unnamed: 0","dx","image_id","lesion_id","dx","dx_type","age","sex","localization"])
print("Shape of X", X.shape)
X.head()

Shape of X (9993, 3072)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,3032,3033,3034,3035,3036,3037,3038,3039,3040,3041,3042,3043,3044,3045,3046,3047,3048,3049,3050,3051,3052,3053,3054,3055,3056,3057,3058,3059,3060,3061,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
0,229,128,132,232,126,128,230,128,127,233,131,133,238,134,141,237,133,141,238,136,145,233,132,135,237,132,139,242,141,157,244,143,156,244,151,160,245,150,160,243,...,147,229,144,142,226,136,132,228,140,137,231,144,138,227,141,130,224,140,129,227,145,133,225,142,128,224,144,130,223,148,136,224,150,139,222,150,138,215,145,133
1,216,122,134,217,120,132,217,117,133,218,120,133,215,116,123,216,118,124,218,122,134,222,127,142,225,133,148,221,128,143,227,134,153,234,143,162,233,143,162,240,...,144,220,138,144,217,136,142,226,144,150,226,144,151,222,143,151,224,146,153,223,147,154,218,143,149,217,142,148,219,144,150,213,141,145,214,140,145,208,136,139
2,245,171,177,246,173,180,247,174,182,247,175,182,248,180,187,248,181,188,248,179,185,248,180,186,248,182,188,248,182,187,248,182,186,248,182,186,248,185,188,248,...,171,236,164,168,233,161,168,229,158,162,228,156,158,225,150,147,223,145,145,221,145,147,215,136,138,215,136,140,213,136,144,213,139,149,217,147,155,215,144,154
3,209,121,135,209,118,134,210,122,137,211,125,140,212,123,139,212,121,138,213,123,140,216,133,146,218,135,151,217,126,143,217,127,138,222,139,148,224,145,156,228,...,116,194,107,116,197,115,127,197,120,130,192,117,127,191,118,126,196,122,127,198,127,127,199,129,127,197,126,121,195,123,119,193,121,118,190,120,120,189,122,125
4,64,28,34,92,45,50,190,129,134,210,151,157,210,141,159,212,147,159,215,161,170,217,160,174,214,150,153,220,169,177,214,158,149,208,140,122,216,153,150,219,...,144,215,146,142,216,150,153,215,150,156,221,167,187,214,152,156,205,123,104,213,149,154,208,144,131,203,126,108,207,135,138,206,137,141,140,85,85,69,30,41


In [40]:
Y = dataset["dx"]
Y = pd.get_dummies(Y)
print("Shape of Y", Y.shape)
Y.head()

Shape of Y (9993, 7)


,akiec,bcc,bkl,df,mel,nv,vasc
0,0,0,0,0,0,1,0
1,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0
4,0,0,0,0,1,0,0


In [25]:
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15, random_state = 42)

print ("number of training examples = " + str(X_train_flat.shape[0]))
print ("number of test examples = " + str(X_test_flat.shape[0]))
print ("X_train_flat shape: " + str(X_train_flat.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test_flat shape: " + str(X_test_flat.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 8494
number of test examples = 1499
X_train_flat shape: (8494, 3072)
Y_train shape: (8494, 7)
X_test_flat shape: (1499, 3072)
Y_test shape: (1499, 7)


In [51]:
X_train = X_train.values
Y_train = Y_train.values

AttributeError: ignored

In [0]:
def LRModel1(input_shape = (3072,), classes = 7):
    X_input = Input(input_shape)
    X = BatchNormalization()(X_input)
    X = Dense(input_shape, activation='sigmoid', name = "sigmoid_activation", kernel_initializer = glorot_uniform(seed=0))(X)
    X = Dense(classes, input_shape=input_shape, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    model = Model(inputs = X_input, outputs = X, name='LogisticRegression')
    return model

def LRModel2(input_shape = (3072,), classes = 7):
    X_input = Input(input_shape)
    X = BatchNormalization()(X_input)
    X = Dense(classes, input_shape=input_shape, activation='sigmoid', name = "sigmoid_activation", kernel_initializer = glorot_uniform(seed=0))(X)
    X = Activation("softmax")(X)
    model = Model(inputs = X_input, outputs = X, name='LogisticRegression')
    return model

In [0]:
model = LRModel2(input_shape = (3072,), classes = 7)

In [0]:
from keras.optimizers import SGD, Adam
adam = Adam(lr= 0.0005)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [49]:
model.fit(X_train, Y_train, epochs = 200, batch_size = 32)

Epoch 1/200
8494/8494 [==============================] - 1s 131us/step - loss: 1.4186 - accuracy: 0.6373
Epoch 2/200
8494/8494 [==============================] - 1s 128us/step - loss: 1.4165 - accuracy: 0.6447
Epoch 3/200
8494/8494 [==============================] - 1s 131us/step - loss: 1.4140 - accuracy: 0.6447
Epoch 4/200
8494/8494 [==============================] - 1s 130us/step - loss: 1.4130 - accuracy: 0.6396
Epoch 5/200
8494/8494 [==============================] - 1s 123us/step - loss: 1.4119 - accuracy: 0.6468
Epoch 6/200
8494/8494 [==============================] - 1s 123us/step - loss: 1.4128 - accuracy: 0.6386
Epoch 7/200
8494/8494 [==============================] - 1s 126us/step - loss: 1.4138 - accuracy: 0.6389
Epoch 8/200
8494/8494 [==============================] - 1s 123us/step - loss: 1.4124 - accuracy: 0.6462
Epoch 9/200
8494/8494 [==============================] - 1s 123us/step - loss: 1.4127 - accuracy: 0.6465
Epoch 10/200
8494/8494 [==============================]

In [0]:
X_test = X_test.values
Y_test = Y_test.values

In [80]:
from sklearn.metrics import confusion_matrix, accuracy_score, average_precision_score, f1_score
y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)
conf_m_test = confusion_matrix(Y_test.argmax(axis=1), y_pred_test.argmax(axis=1))
y_test_precision = average_precision_score(Y_test, y_pred_test)
y_test_acc1 = accuracy_score(Y_test.argmax(axis=1), y_pred_test.argmax(axis=1))
y_test_f1 = f1_score(Y_test.argmax(axis=1), y_pred_test.argmax(axis=1), average = 'weighted')
y_train_f1 = f1_score(Y_train.argmax(axis=1), y_pred_train.argmax(axis=1), average = 'weighted')
print("Precision of the Logistic Regression model in test set: " + str(y_test_precision) + "\n Accuracy of the Logistic Regression Modelin test set: " + str(y_test_acc1))
print("f1 of the Logistic Regression model in test set: " + str(y_test_f1))

Precision of the Logistic Regression model in test set: 0.25005542865124253
 Accuracy of the Logistic Regression Modelin test set: 0.6490993995997332
f1 of the Logistic Regression model in test set: 0.6512995453824352


In [54]:
y_train_precision = average_precision_score(Y_train, y_pred_train)
y_train_acc1 = accuracy_score(Y_train.argmax(axis=1), y_pred_train.argmax(axis=1))
print("Precision of the Logistic Regression model in test set: " + str(y_test_precision) + "\n Accuracy of the Logistic Regression Modelin test set: " + str(y_test_acc1))

Precision of the Logistic Regression model in test set: 0.25005542865124253
 Accuracy of the Logistic Regression Modelin test set: 0.6490993995997332


In [70]:
from keras.models import Sequential
from keras.layers import Dropout
# Set CNN model
# Our system of layers => [[Conv2D -> relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out
input_shape = (32, 32, 3)
num_classes = 7


model_cnn = Sequential()
model_cnn.add(Conv2D(10, kernel_size = (3, 3), activation = 'relu', padding = 'Same', input_shape = input_shape))
model_cnn.add(MaxPooling2D(pool_size = (2, 2)))
model_cnn.add(Dropout(0.25))
model_cnn.add(Conv2D(20, kernel_size = (5, 5), activation = 'relu', padding = 'Same'))
model_cnn.add(MaxPooling2D(pool_size = (3, 3)))
model_cnn.add(Dropout(0.25))
model_cnn.add(Conv2D(30, kernel_size = (7, 7), activation = 'relu', padding = 'Same'))
model_cnn.add(MaxPooling2D(pool_size = (4, 4)))
model_cnn.add(Dropout(0.25))

model_cnn.add(Flatten())
model_cnn.add(Dense(120, activation='relu'))
model_cnn.add(Dropout(0.2))
model_cnn.add(Dense(64, activation ='relu'))
model_cnn.add(Dropout(0.2))
model_cnn.add(Dense(num_classes, activation='softmax'))
model_cnn.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 32, 32, 10)        280       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 16, 16, 10)        0         
_________________________________________________________________
dropout_31 (Dropout)         (None, 16, 16, 10)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 16, 16, 20)        5020      
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 5, 5, 20)          0         
_________________________________________________________________
dropout_32 (Dropout)         (None, 5, 5, 20)          0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 5, 5, 30)        

In [0]:
model_cnn.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
X_train3D = np.reshape(X_train, (X_train.shape[0], 32, 32, 3))
X_test3D = np.reshape(X_test, (X_test.shape[0], 32, 32, 3))

In [73]:
model_cnn.fit(X_train3D, Y_train, batch_size = 32, epochs = 200)

Epoch 1/200
8494/8494 [==============================] - 11s 1ms/step - loss: 0.9855 - accuracy: 0.6658
Epoch 2/200
8494/8494 [==============================] - 11s 1ms/step - loss: 0.9619 - accuracy: 0.6647
Epoch 3/200
8494/8494 [==============================] - 11s 1ms/step - loss: 0.9368 - accuracy: 0.6687
Epoch 4/200
8494/8494 [==============================] - 11s 1ms/step - loss: 0.9289 - accuracy: 0.6668
Epoch 5/200
8494/8494 [==============================] - 11s 1ms/step - loss: 0.9283 - accuracy: 0.6715
Epoch 6/200
8494/8494 [==============================] - 11s 1ms/step - loss: 0.9160 - accuracy: 0.6774
Epoch 7/200
8494/8494 [==============================] - 11s 1ms/step - loss: 0.9185 - accuracy: 0.6807
Epoch 8/200
8494/8494 [==============================] - 11s 1ms/step - loss: 0.8951 - accuracy: 0.6865
Epoch 9/200
8494/8494 [==============================] - 11s 1ms/step - loss: 0.8898 - accuracy: 0.6842
Epoch 10/200
8494/8494 [==============================] - 11s 1m

In [79]:
from sklearn.metrics import confusion_matrix, accuracy_score, average_precision_score, f1_score
y_pred_test = model_cnn.predict(X_test3D)
y_pred_train = model_cnn.predict(X_train3D)
conf_m_test = confusion_matrix(Y_test.argmax(axis=1), y_pred_test.argmax(axis=1))
y_test_precision = average_precision_score(Y_test, y_pred_test)
y_test_acc1 = accuracy_score(Y_test.argmax(axis=1), y_pred_test.argmax(axis=1))
y_test_f1 = f1_score(Y_test.argmax(axis=1), y_pred_test.argmax(axis=1), average = 'weighted')
y_train_f1 = f1_score(Y_train.argmax(axis=1), y_pred_train.argmax(axis=1), average = 'weighted')
print("Precision of the Logistic Regression model in test set: " + str(y_test_precision) + "\n Accuracy of the Logistic Regression Modelin test set: " + str(y_test_acc1))
print("f1 of the Logistic Regression model in test set: " + str(y_test_f1))

Precision of the Logistic Regression model in test set: 0.3878177126603025
 Accuracy of the Logistic Regression Modelin test set: 0.7084723148765844
f1 of the Logistic Regression model in test set: 0.6871472371918573
